In [32]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import json
from lxml import html

df = pd.read_csv('data_linkcompany_test.csv')
# get links
links = df['link']
# create field 
df['company_name'] = ''
df['company_address'] = ''
df['company_size'] = ''
df['company_website'] = ''
df['company_type'] = ''
df['company_details'] = ''

In [5]:
def get_company_name(soup):
    div_tag = soup.find('div', {'class': 'content'})
    job_name = div_tag.find('h1', {'class': 'name'}).get_text()

    return job_name

In [8]:
def get_company_address(soup):
    div_tag = soup.find('div', {'class': 'content'})
    p_tag = div_tag.find('p')
    company_address = p_tag.get_text()
    
    return company_address

In [28]:
def get_company_size(soup):
    span_tag = soup.find('span', {'class': 'mdi mdi-account-supervisor'})
    if span_tag: 
        li_tag = span_tag.parent
        company_size = li_tag.get_text().strip()
        clean = re.sub(r'\s+', ' ', company_size)
        company_size = clean
    else: company_size = ""
    
    return company_size

In [25]:
def get_company_website(soup):
    span_tag = soup.find('span', {'class': 'mdi mdi-link'})
    if span_tag: 
        li_tag = span_tag.parent
        company_link = li_tag.get_text().strip()
        clean = re.sub(r'\s+', ' ', company_link)
        company_link = clean
    else: company_link = ""

    return company_link

In [26]:
def get_company_type(soup):
    span_tag = soup.find('span', {'class': 'mdi mdi-gavel'})   
    if span_tag: 
        li_tag = span_tag.parent
        company_type = li_tag.get_text().strip()
        clean = re.sub(r'\s+', ' ', company_type)
        company_type = clean
    else: company_type = ""

    return company_type

In [30]:
def get_company_details(soup):
    div_tag = soup.find('div', {'class': 'intro-section intro-section-1'})
    tag = div_tag.find('div', {'class': 'main-text'})
    company_details = tag.get_text(separator='\n', strip=True)
    
    return company_details

In [33]:
# Khởi tạo trình duyệt Edge
driver = webdriver.Edge()
i = 0
# Duyệt qua từng link và lấy thông tin
for link in links:
    # Mở trang web
    driver.get(link)

    # Chờ cho trang web tải xong
    driver.implicitly_wait(10)

    # Lấy HTML của trang web
    html = driver.page_source

    # Phân tích HTML bằng BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    #find company name
    company_name = get_company_name(soup)
    df['company_name'][i] = company_name
    
    #find company address
    company_address = get_company_address(soup)
    df['company_address'][i] = company_address
    
    #find company size
    company_size = get_company_size(soup)
    df['company_size'][i] = company_size
    
    #find company type
    company_type = get_company_type(soup)
    df['company_type'][i] = company_type
    
    #find company website
    company_website = get_company_website(soup)
    df['company_website'][i] = company_website
    
    #find company details
    company_details = get_company_details(soup)
    df['company_details'][i] = company_details
    
    print(str(i)+ " done")
    i = i + 1
    df.to_csv('data_company.csv', index=False)
# Đóng trình duyệt
driver.quit()


0 done
1 done


# Convert csv to json

In [ ]:
# Đọc dữ liệu từ file CSV hoặc thực hiện các bước để có DataFrame của bạn
dfr = pd.read_csv('data_company.csv')
# Chuyển đổi DataFrame thành danh sách bản ghi
records = dfr.to_dict(orient='records')

# Chuyển đổi dữ liệu ngày thành định dạng chuẩn (ví dụ: ISO 8601)
for record in records:
    if 'company_size' in record and pd.isna(record['company_size']):
        record['company_size'] = None
    if 'company_type' in record and pd.isna(record['company_type']):
        record['company_type'] = None
    if 'company_website' in record and pd.isna(record['company_website']):
        record['company_website'] = None

# Ghi vào file JSON mà không sử dụng Unicode Escape
with open('company_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(records, json_file, ensure_ascii=False, indent=2)
# Lưu DataFrame thành file JSON
#dfr.to_json('test.json', orient='records', lines=True)